# KSI monitoring: zpětná vazba na úlohy - text
V tomto souboru se nachází textová zpětná vazba řešitelů ke konkrétním úlohám.

In [7]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns

import util
from util.year import year as current_year
from db import session
import model
from datetime import datetime
import json

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [8, 6]
print(datetime.now())

ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

In [ ]:
Feedback = namedtuple(
    'Feedback',
    ['user', 'task', 'comment']
)

feedbacks_raw = session.query(
    model.Feedback,
    model.Task,
).\
    join(model.Feedback.r_task).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Feedback.r_user).\
    filter(model.User.role == 'participant').\
    order_by(model.Task.id).all()

feedbacks = []
for f, task in feedbacks_raw:
    data = {
        item['id']: item['answer']
        for item in json.loads(f.content)
    }
    feedbacks.append(Feedback(
        f.user,
        task,
        data['comment'] if 'comment' in data else None,
    ))

In [ ]:
large_tasks = util.task.large_tasks().all()

In [ ]:
counts = {}

unique_tasks = set()

for feedback in feedbacks:
    unique_tasks.add(feedback.task)

for task in unique_tasks:
    # print("----", task)
    feedbacks_of_this_task = list(filter(lambda x: 
                                         x.task == task and
                                         x.comment and
                                         len(x.comment) > 0,
                                     feedbacks))
    
    if len(feedbacks_of_this_task) == 0:
        continue
    
    display(Markdown('## {name}'.format(name=task)))
    for feedback in feedbacks_of_this_task:
        display(feedback.comment)
